## Post Import in Group

This demonstrates the process of deploying a new Power BI Model as a PBIX file to a Power BI Workspace.  In this project we've provided two sample PBIX files to illustrate these concepts:
- files/AdventureWorks.pbix - this is a fairly small model built on the AdventureWorksDW database
- files/NYCYellow.pbix - this is a large mode that is over 1GB used to illustrate the asynch process of loading large models via a temporary blob location.  Note that since this file exceeds 1GB it can only be hosted in a workspace with a premium capacity.

The general flow of this process is:
1. Authenticate the Service Principal.
1. Open the local file to determine its size.
1. If the file is less than 1GB, read the file as a binary multi-part encoded stream, and call the Imports API for the workspace.
1. If the file is larger than 1GB, call the Imports api to generate a temporary upload location, use the blob API to upload the file, and call the Imports API passing in the path to the temporary blob location.

The documentation for the relevant APIs can be found here:
- https://learn.microsoft.com/en-us/rest/api/power-bi/imports/create-temporary-upload-location-in-group
- https://learn.microsoft.com/en-us/rest/api/power-bi/imports/post-import-in-group

In [ ]:
pip install requests msal jsons azure-storage-blob requests_toolbelt

In [ ]:
#For orgaization purposes I put notbooks in subfolders not the root of the proejct.aad_token
#This code adds the root directory of the project to the sys path so we can load class modules from the services folder
#I think this only needs to be run once, but including it for completeness.
import os, sys
projectRoot = os.path.abspath('.')
directory = os.path.dirname(projectRoot)
if not directory in sys.path: sys.path.append(directory)

In [ ]:
workspaceId="16453fea-cb0f-4ef4-b1b2-bede9d3b92be"
displayName="AdventureWorks.pbix"

In [ ]:
#This leverages the code encapsulated in services/aadservice.py that encapsulates the service principle login
from services.aadservice import AadService
aadToken = AadService.get_access_token()

#we don't prebuild the header like in other examples becuase the reqeusts have different content types

In [ ]:
from os import path
#file_location = 'files/AdventureWorks.pbix'
fileLocation = f'files/{displayName}'
fileSize = path.getsize(fileLocation)


In [ ]:
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

#This demonstrates loading files under 1GB
if fileSize < 1073741824:
    print("File is less than 1GB this cell is loading the file as form data")

    url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/imports?datasetDisplayName={displayName}'
    headers = {
        'Content-Type': 'multipart/form-data',
        'authorization': 'Bearer ' + aadToken
    }

    # you need this dictionary to convert a binary file into form-data format
    # None here means we skip the filename and file content is important 
    files = {'value': (None, open(fileLocation, 'rb'), 'multipart/form-data')}

    mp_encoder = MultipartEncoder(fields=files)
    print(mp_encoder)
    r = requests.post(
        url=url,
        data=mp_encoder,  # The MultipartEncoder is posted as data, don't use files=...!
        # The MultipartEncoder provides the content-type header with the boundary:
        headers=headers
    )
else:
    print("File is larger than 1GB this cell did nothing")


In [ ]:
import requests
import json
from azure.storage.blob import BlobClient

#This demonstrates loading files under 1GB
if fileSize >= 1073741824:
    tempUrl = ""

    #Part 1:  Request a temporary blob location from the API.
    apiUrl = f'https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/imports/createTemporaryUploadLocation'
    headers =  {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + aadToken
    } 

    apiResponse = requests.post(apiUrl, headers=headers)
    #error handling for createTemporaryUplodadLocation
    if apiResponse.status_code != 200:
        description = f'Error requesting temp location:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{api_response.headers.get("RequestId")}\n  -Text:\t{api_response.text}'
        print(description)
    else:
        api_response = json.loads(apiResponse.text)
        tempUrl = api_response['url']

    #Part 2:  Now that we have the temporary location, upload the file to it via the BlobClient
    if tempUrl != '':
        client = BlobClient.from_blob_url(tempUrl)
        with open(fileLocation, 'rb') as data:
            client.upload_blob(data)
        print(f"{fileLocation} uploaded")
    else:
        print("No sas url specified.")

    #Part 3:   Call the import API with the temp location as 'fileUrl' in the body of the API call
    apiUrl = f"https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/imports?datasetDisplayName={displayName}&nameConflict=Ignore&skipReport=false&overrideReportLabel=true&overrideModelLabel=true"

    body = {
        "fileUrl": tempUrl
    }

    apiResponse = requests.post(
        url=apiUrl,
        headers=headers,
        data=json.dumps(body)
    )

    if apiResponse.status_code != 200:
        description = f'Error importing file:\n  -Status Code:\t{apiResponse.status_code}\n  -Reason:\t{apiResponse.reason}\n  -RequestId:\t{apiResponse.headers.get("RequestId")}\n  -Text:\t{apiResponse.text}'
        print(description)
    else:
        apiResponse = json.loads(apiResponse.text)
        print(json.dumps(apiResponse,indent=2))
